In [49]:
from remove_duplicates import *
%reload_ext autoreload
%autoreload 2

In [81]:
org_df = None
with pymysql.connect(**CONN_INFO) as cursor:
        query = """SELECT oid, name
                   FROM Organizations"""
        cursor.execute(query)

        org_lists = [[oid, name] for oid, name in cursor]
        org_df = pd.DataFrame(org_lists, columns=['oid', 'name'])


In [24]:
# org_df = org_df.sample(1000)

In [82]:
org_df['normed_name'] = org_df['name'].apply(norm_name)
org_df['hard_coded_name'] = org_df['normed_name'].apply(hard_coded_matches)

In [ ]:
cpy_org_df = org_df.copy()

In [ ]:
org_df['matched_indices'] = org_df.apply(lambda row: match_other_rows(row, cpy_org_df), axis=1)

In [ ]:
# Yes, this needs to happen twice
cpy_org_df = org_df.copy()

In [ ]:
org_df['delete'] = org_df.apply(lambda row: mark_rows_for_deletion(row, cpy_org_df), axis=1)

In [ ]:
cpy_org_df = org_df.copy()

In [ ]:
org_df = org_df[~org_df['delete']]

In [43]:
# The following code is used for making your duplicate spreadsheet
org_df = org_df[org_df['matched_indices'].apply(len) > 0]

In [67]:
output = org_df.apply(lambda row: join_dup_org_names(row, cpy_org_df), axis=1)

In [69]:
mx = org_df['matched_indices'].apply(len).max()

In [74]:
columns = ['name',
           'hard_coded_name',
           'normed_name',
           'oid']
for i in range(1, mx+1):
    columns.append('name_' + str(i))
    columns.append('oid_' + str(i))

In [76]:
output = output[columns]

In [80]:
output.to_csv('duplicate_removal_final.csv', index=False)

In [13]:
%%timeit
d = org_df['name'].apply(lambda name: l_distance(name, 'Sierra Club'))

10 loops, best of 3: 126 ms per loop
